In [1]:
#http://boto3.readthedocs.io/en/latest/guide/sqs-example-using-queues.html
#http://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/Welcome.html

In [2]:
import boto3

In [3]:
sqs = boto3.client('sqs')

In [8]:
# Standard Queues vs Fifo queues
# Suffix the queue name with .fifo if you are looking for FIFO queue
sqs.create_queue(
    QueueName='mulani-queue',
    Attributes={
        'DelaySeconds':'60',
        'MessageRetentionPeriod':'86400'
    })

{'QueueUrl': 'https://ap-south-1.queue.amazonaws.com/088045063209/mulani-queue',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '335',
   'content-type': 'text/xml',
   'date': 'Sun, 26 Nov 2017 14:12:20 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'd536282f-395b-58d5-949e-7abbf75f1fec'},
  'HTTPStatusCode': 200,
  'RequestId': 'd536282f-395b-58d5-949e-7abbf75f1fec',
  'RetryAttempts': 0}}

In [12]:
response = sqs.list_queues()

In [14]:
response['QueueUrls']

['https://ap-south-1.queue.amazonaws.com/088045063209/mulani-queue']

In [16]:
sqs.get_queue_url(QueueName='mulani-queue')

{'QueueUrl': 'https://ap-south-1.queue.amazonaws.com/088045063209/mulani-queue',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '335',
   'content-type': 'text/xml',
   'date': 'Sun, 26 Nov 2017 14:13:29 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'e2646d51-9b00-54f0-88cc-3fc55549d3c9'},
  'HTTPStatusCode': 200,
  'RequestId': 'e2646d51-9b00-54f0-88cc-3fc55549d3c9',
  'RetryAttempts': 0}}

In [ ]:
sqs.delete_queue(QueueName='mulani-queue')

In [20]:
sqs.set_queue_attributes(
    QueueUrl=sqs.get_queue_url(QueueName='mulani-queue')['QueueUrl'],
    Attributes={
        'VisibilityTimeout':'60' #message once received will not be visible for other consumers for 5 sec during which it can be deleted by primary consumer
    }
)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '225',
   'content-type': 'text/xml',
   'date': 'Sun, 26 Nov 2017 15:16:35 GMT',
   'server': 'Server',
   'x-amzn-requestid': '851ffded-4c61-5c4f-acfe-af799172bd78'},
  'HTTPStatusCode': 200,
  'RequestId': '851ffded-4c61-5c4f-acfe-af799172bd78',
  'RetryAttempts': 0}}

In [22]:
response = sqs.send_message(
    QueueUrl=sqs.get_queue_url(QueueName='mulani-queue')['QueueUrl'],
    DelaySeconds=10, #message visible for receipt only after 10 seconds
    MessageAttributes={
        'Author':{
            'DataType':'String',
            'StringValue' : 'Manish Mulani'
        },
        'Title':{
            'DataType':'String',
            'StringValue':'Hello World'
        },
        'Rank':{
            'DataType':'Number',
            'StringValue' : '6'
        }
    },
    MessageBody='Come on man. Thats the spirit'
)

In [24]:
response['MessageId']

'699cb578-dc1c-495e-af89-82f4b918c1f8'

In [35]:
response = sqs.receive_message(
    QueueUrl=sqs.get_queue_url(QueueName='mulani-queue')['QueueUrl'],
    AttributeNames=['SentTimestamp'],
    MaxNumberOfMessages=1,
    MessageAttributeNames=['All']
)

In [36]:
response

{'Messages': [{'Attributes': {'SentTimestamp': '1511709426400'},
   'Body': 'Come on man. Thats the spirit',
   'MD5OfBody': '2fdf26b8e05f4cdbc98239ccf1adeb33',
   'MD5OfMessageAttributes': 'f78960b4bf9c67a10d9f8204bce95537',
   'MessageAttributes': {'Author': {'DataType': 'String',
     'StringValue': 'Manish Mulani'},
    'Rank': {'DataType': 'Number', 'StringValue': '6'},
    'Title': {'DataType': 'String', 'StringValue': 'Hello World'}},
   'MessageId': '699cb578-dc1c-495e-af89-82f4b918c1f8',
   'ReceiptHandle': 'AQEBfaUKwmwQ1nNeH+0XiC+djn1HxFLou0AqbuTGHrL9FXnzqpSRwJLOmBXQA/CcGuJrmOLcBmouh+BXut7PR6gukzGWA6LI0i0mFqZHTYdoL7sqUJQRjEhp8+Aju4N7AFTmNxEcOxTUZ8XF1v3E5LuP7G4GYs8lVGv0Jp3/sOqvWRbAxsoEORaTGraEBL2LWcCAWvEddAaWlhgDQ64Rj0fjWjdtyK3njRC0k9+Ohh8sUkThIuP/fVNYQG0GV2mLgPcQ7jQF3P/g/gxXZ8TgeX7liLv+EOH55HCutGYx/+M/x/zSdWA/3a9G3w0gcdlLxdDaAiyQqV+FcGpZbEzgn1eYfKmTRIcOzD6sU8FyGduduUtrp8QkAqMe5eK14l6Wo0BKMApYJyfuAfqXH1D0MOH3+w=='}],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-al

In [39]:
handleToDelete = response['Messages'][0]['ReceiptHandle']

In [40]:
# Message stays in the Queue until its deleted
sqs.delete_message(
    QueueUrl=sqs.get_queue_url(QueueName='mulani-queue')['QueueUrl'],
    ReceiptHandle=handleToDelete
)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '215',
   'content-type': 'text/xml',
   'date': 'Sun, 26 Nov 2017 15:28:59 GMT',
   'server': 'Server',
   'x-amzn-requestid': '181ddfc1-01db-5c2f-b77e-8d55de2080b4'},
  'HTTPStatusCode': 200,
  'RequestId': '181ddfc1-01db-5c2f-b77e-8d55de2080b4',
  'RetryAttempts': 0}}